In [6]:
import cudaq
import numpy as np
from itertools import product
from cudaq import spin

In [7]:
# Define Pauli matrices
paulis = {
    'I': np.array([[1, 0], [0, 1]], dtype=complex),
    'X': np.array([[0, 1], [1, 0]], dtype=complex),
    'Y': np.array([[0, -1j], [1j, 0]], dtype=complex),
    'Z': np.array([[1, 0], [0, -1]], dtype=complex)
}

p_spin_op = {
    'I': spin.i,
    'X': spin.x,
    'Y': spin.y,
    'Z': spin.z
}
pauli_labels = list(paulis.keys())

def get_pauli_basis(n_qubits):
    """Generate tensor products of Pauli operators for n qubits."""
    basis = []
    labels = []
    for label in product(pauli_labels, repeat=n_qubits):
        op = paulis[label[0]]
        for l in label[1:]:
            op = np.kron(op, paulis[l])
        basis.append(op)
        labels.append(''.join(label))
    return labels, basis

def decompose_into_paulis(A):
    """Decompose Hermitian matrix A into Pauli basis."""
    n = int(np.log2(A.shape[0]))
    labels, basis = get_pauli_basis(n)
    coeffs = []
    for label, P in zip(labels, basis):
        # Coefficient: Tr(P†A) / 2^n (note P† = P for Pauli)
        coeff = np.trace(P.conj().T @ A) / (2 ** n)
        if not np.isclose(coeff, 0, atol=1e-10):
            coeffs.append((coeff, label))
    return coeffs

def to_spin_operator(coeffs, labels):
    operator = 0.0*spin.i(0)
    for i in range(1, len(labels[0])):
        operator *= spin.i(i)
    
    for i in range(len(labels)):
        ops = coeffs[i]
        for j in range(len(labels[i])):
            ops *= p_spin_op[labels[i][j]](j)
        operator += ops
    return operator
            
to_spin_operator([2.0, 1.0], ["II", "IZ"])      